In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

import requests as re
from html.parser import HTMLParser

sns.set_context('poster')
sns.set_style('whitegrid')

In [11]:
df = pd.read_json('./rawData/train_data.json')

In [12]:
df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = [c.replace('-', '_') for c in df.columns]
df.columns = [c.replace('/', '_') for c in df.columns]

In [13]:
df['bedrooms'] = df['bedrooms'].apply(float)
df['bedrooms'].fillna(0, inplace=True)
df['bathrooms'].fillna(0, inplace=True)

In [14]:
df.drop('index', axis=1, inplace=True)
df['interestVal'] = df['interest_level'].map({'high': 1, 'medium': 0.5, 'low':0})

In [15]:
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

In [16]:
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [17]:
def descrClean(x):
    des = strip_tags(x)
    return des.lower()

In [18]:
df['description'] = df['description'].apply(descrClean)

In [19]:
df.to_json('./rawData/cleaned.json')